In [1]:
# installs

#!{sys.executable} -m pip install tqdm

# ниже - неудачная попытка отрисовать в three.js
#!curl -sL https://deb.nodesource.com/setup_12.x | sudo -E bash -
#!sudo apt update
#!sudo apt install -y nodejs
#!{sys.executable} -m pip install nodejs
#!{sys.executable} -m pip install pythreejs
# теперь можно слева во вкладе extensions установить jupyterlab-manager и threejs

In [2]:
# imports
import sys
import tqdm
from collections import Counter
import itertools

In [5]:
N = ToricLattice(7)
K = N([-3, 1, 1, 1, 1, 1, 1]) # canonical divisor
E = identity_matrix(QQ, 7)[:,1:].columns() # E is the set of exceptional curves, so the basis is L, E[0], .. , E[5]
Line = lambda exceptionals : (-K + sum(exceptionals)) / 3
L = Line(E)



Q = diagonal_matrix([1]+6*[-1])
dot = lambda a, b: a*Q*b
gram_matrix = lambda rays: matrix([[dot(a,b) for a in rays] for b in rays])

minus_one_curves = E + [L-ei-ej for ei,ej in itertools.combinations(E, 2)] + [-K-L+e for e in E]

# take one Fujita face for each isomorphism class (determined by CPW, may check explicitly later)
# and turn them into subcones in Mori cone by adding -K
Fujita_cone_examples = {label:Cone(rays+[-K]) for label,rays in {
        'B6': E,
        'B5': E[:5],
        'B5P1': E[:4]+[L-E[4]-E[5]], # B(5) for P1xP1. Another type of independent set of 5 (-1)-curves, this one is maximal
        'B4': E[:4],
        'B3': E[:3],
        'B2': E[:2],
        'B1': E[:1],
        'C6': E[:5]+[L-E[5]-e for e in E[:5]], # C(6). Schlafli graph is 4-ultrahomogeneous, and, after fixing two pairs, three others are defined uniquely
    }.items()}

is_covered_by = lambda cone1, cone2: all(cone2.contains(ray) for ray in cone1.rays()) and cone2.interior_contains(sum(cone1.rays()))

def independent_sets(vectors, size = 6):
    if size == 0:
        yield []
    for i, v in enumerate(vectors):
        orthogonals = [v2 for v2 in vectors[i+1:] if dot(v, v2)==0]
        for subset in independent_sets(orthogonals, size-1):
            yield subset + [v]

#cylinder1 = lambda E : E + [Line(E)-E[-1], -K-Line(E)+E[-1]]   
def cylinder2(E, e):
    '''
    E is a list of exceptional curves, and e is one of them
    returns rays of the cone of divisors H such that the cylinder (P^2 minus conic through E-e and tangent line through e) is H-polar
    '''
    tangent = Line(E) - E[-1]
    conic = - K - tangent
    return E+[tangent,conic]

def cylinder1(E,e):
    """
    cylinder complement to E and lines through blowdowns of e and one of E-e
    """
    return E+[Line(E)-e-f for f in E if f!=e]



def cylinder_cones(cylinder=cylinder2):
    '''
    returns a list of cones of compatible divisors for cylinders of considered type
    '''
    for blowdown in independent_sets(minus_one_curves):
        for e in blowdown:
            yield Cone(cylinder(blowdown,e))


In [7]:
# setting cones

cones = [cone for cone in cylinder_cones()]
effective_cone = Cone(minus_one_curves)
ample_cone = Cone([Q*ray for ray in effective_cone.dual().rays()])

minkowski_sum = lambda cones : Cone([ray for cone in cones for ray in cone.rays()])
contains_ample = lambda cone : all(cone.contains(ray) for ray in ample_cone.rays())
is_subcone = lambda cone1, cone2 : all(cone2.contains(ray) for ray in cone1.rays())


In [8]:
# studying bad ray
N = ToricLattice(7)
badray = N(51, -26, -13, -7, -4, -2, -1)
print(effective_cone.contains(badray))
for face in effective_cone.faces()[6]:
    intersection = face.intersection(Cone([ray,K]))
    if intersection.dim() == 0:
        continue
    else:
        break
print(intersection.rays())
print(face.rays())
ray = intersection.rays()[0]
print(face.relative_interior_contains(ray))
e1, e2, e3, e4, e5, f1 = face.rays()[:6]
b = e1 + f1
R = 63*e1 + 59*e2 + 54*e3 + 50*e4 + 19*e5
print(R)
any(cone.interior_contains(badray) for cone in cones)
# its Fujita face is spanned by rays L-E1-Ej and L-\sum(Ei)+Ej, where j=2..6, the class K+\mu*H equals ray/12.
# they comprise 5 pairs 
# its Fujita map is of type C(6)

True


NameError: name 'ray' is not defined

In [ ]:

# вычисляем типы граней
def face_type(face):
    m = gram_matrix(face.rays())
    eigenvalues = Counter(m.eigenvalues())
    if eigenvalues[-2]==5 and eigenvalues[0]==5:
        return "C"
    if len(eigenvalues.keys())!=1:
        print('ERROR face_type, eigenvalues: ',eigenvalues)
    return f"B{eigenvalues[-1]}"

Fujita_face = lambda dim, num: effective_cone.faces()[dim][num]
face_of_type = {
    'B6': Fujita_face(6,-1),
    'B5': Fujita_face(5,-1), # в B(5) два разных случая: набор из 5 непересекающихся -1-прямых может быть максимальным
    'B4': Fujita_face(4,-1),
    'B3': Fujita_face(3,-1),
    'B2': Fujita_face(2,-1),
    'B1': Fujita_face(1,-1),
    'C': Fujita_face(6,0),
}
assert face_type(face_of_type["C"]) == "C"
assert face_type(face_of_type["B6"]) == "B6"


Fujita_face_examples = [
    
]

cone_of_type = {label : Cone(list(face.rays())+[-K]) for label, face in face_of_type.items()}



In [ ]:
ample_cone.dual().rays()
symmetries = [tuple(1 if i==j else -1 if i==j+1 else 0 for j in range(7)) for i in range(2,7)]
ample_segment = Cone(list(ample_cone.dual().rays())+symmetries).dual()

In [ ]:
# dissections 

dissecting_planes = lambda cones : [ray for cone in cones for ray in cone.dual().rays() if not ample_segment.dual().contains(ray)]

def dissect_with_plane(cones, plane):
    dissected = []
    for cone in cones:
        #print(f'dissecting with plane, {len(dissected)} cones')
        if cone.dual().contains(plane) or cone.dual().contains(-plane):
            dissected.append(cone)
        else:
            dual_rays = list(cone.dual().rays())
            positive_part = Cone(dual_rays + [plane]).dual()
            negative_part = Cone(dual_rays + [-plane]).dual()
            dissected.append(positive_part)            
            dissected.append(negative_part)            
    return dissected

def dissect_with_planes(cones, planes):
    dissected = cones
    for i,p in enumerate(planes):
        print(f'dissecting with {i}-th plane of {len(planes)}, {len(dissected)} cones')
        dissected = dissect_with_plane(dissected, p)
    return dissected

dissects = lambda cone, plane: not( cone.dual().contains(plane) or cone.dual().contains(-plane) )


def halves(cone, plane):
    dual_rays = list(cone.dual().rays())
    negative_part = Cone(dual_rays + [-plane]).dual()
    positive_part = Cone(dual_rays + [plane]).dual()
    return negative_part, positive_part

def dissect_wrt(dissected_cone, dissecting_cone): # intersection is returned separately
    parts = []
    remainder = dissected_cone
    for plane in dissecting_cone.dual().rays():
        if not dissects(remainder, plane):
            continue
        negative_part, positive_part = halves(remainder, plane)
        parts.append(negative_part)
        remainder = positive_part
    if remainder.intersection(dissecting_cone).is_solid():
        return parts, remainder
    else:
        parts.append(remainder)    
        return parts, None

#[c.rays() for c in dissect_wrt(Cone([[1,0],[0,1]]),Cone([[1,1],[-1,1]]))]

class Pseudofan: # solid version
    def __init__(self, cone):
        self.cones = [cone]
        #for cone in cones:
        #    self.append(cone)
    
    def remove(self, cone):
        #print('removing cone\n',cone.rays())
        outer_parts = [dissect_wrt(old, cone)[0] for old in self.cones]
        #print('outer parts\n', outer_parts)
        self.cones  = list(itertools.chain(*outer_parts))
        #print('cones\n', self.cones)

    def intersect(self, cone):
        intersections = [cone.intersection(old) for old in self.cones]
        self.cones = [c for c in intersections if c.is_solid()]


def remove_cones(pseudofan, cones_list, cap = None):
    pbar = tqdm(cones_list)
    for cone in pbar:
        pseudofan.remove(cone)
        pbar.set_description(f'{len(pseudofan.cones)} cones in pseudofan ')
        if cap:
            P.cones = P.cones[:cap]

In [ ]:
C1, C2 = Cone([[1,0],[0,1]]), Cone([[1,1],[-1,1]])
#F1, F2 = Fan([C1]), Fan([C2])
#F1.common_refinement(F2)
P = Pseudofan(C2)
P.remove(C1)
P.remove(C2)
for c in P.cones:
    print(c.rays())    

In [ ]:
P = Pseudofan(ample_segment)        
remove_cones(P, filtered_cones)
remove_cones(P, cones[::-1], cap=100)
Counter([c.dim() for c in P.cones])
bad_rays = [sum(c.rays()) for c in P.cones]
pbar = tqdm(bad_rays)
#pbar.set_description("Processing %s" % char)
for ray in pbar:
    if not any(c.contains(ray) for c in cones):
        print(ray)

In [ ]:
N = ToricLattice(3)
e1, e2, e3 = N([1,0,0]), N([0,1,0]), N([0,0,1])
r = e1+e2+e3
P3 = Pseudofan(Cone([e1,e2,e3]))
P3.remove(Cone([r,e2,e3]))
P3.remove(Cone([e1,r,e3]))
P3.remove(Cone([e1,r,r]))
for c in P3.cones:
    print(c.rays())
    #c.plot().show()

In [ ]:
#print(cone_of_type['B6'].rays())
[cone.rays() for cone in cones if is_subcone(cone_of_type['B5'],cone)]

In [ ]:
curves = [E[1], E[2], L-E[0]-E[1], L-E[0]-E[2]]
[c for c in minus_one_curves() if all(dot(c,e)==0 for e in curves)]

In [ ]:
is_covered_by = lambda cone1, cone2: all(cone2.contains(ray) for ray in cone1.rays()) and cone2.interior_contains(sum(cone1.rays()))
#[cone.rays() for cone in cones if Fujita_cone_examples['B5P1'].intersection(cone).dim()]
remove_cones(Pseudofan(Fujita_cone_examples['B5P1']), cones)


In [ ]:
A=Fujita_cone_examples['B5P1']
A.intersection(A)
A.dim()
